# Imports

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import re

import pandas as pd
import numpy as np

# Helper Functions

In [150]:

def get_naics(
    url= "https://www.dnb.com/business-directory/company-information.outpatient_care_centers.ca.html?page=2", 
    driver_path="/home/ada/Downloads/geckodriver"
    ):

    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = False
    driver = webdriver.Firefox(capabilities=cap, executable_path=driver_path)
    driver.get(url)

    vals = driver.find_element_by_xpath(
        "//div[@class='codes-title naics']/span").get_attribute("innerHTML")

    vals = vals.split("&")[0]
    print(vals)

    driver.close()

    return vals



def get_dnd_links(
    url = "https://www.dnb.com/business-directory/company-information.health_care_and_social_assistance.ca.html?page=2",
    driver_path = "/home/ada/Downloads/geckodriver"
):


    # company dnd links
    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = False
    driver = webdriver.Firefox(capabilities=cap, executable_path=driver_path)

    driver.get(url)

    elems = driver.find_elements_by_xpath("//a[@href]")
    links = []
    for elem in elems:
        links.append(elem.get_attribute("href"))

    driver.close()

    links = [i for i in links if "company-profiles" in i]

    return links

    
def get_company_details(
    dnd_url = "https://www.dnb.com/business-directory/company-profiles.alberta_health_services.129b1f80ef8deb9b06ed739f80223ccb.html",
    driver_path = "/home/ada/Downloads/geckodriver"
):


    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = False
    driver = webdriver.Firefox(capabilities=cap, executable_path=driver_path)
    driver.get(dnd_url)

    # address
    address = driver.find_element_by_xpath(
        "//div[@class='col-md-11']/span[@class='company_data_point' and @name='company_address']/span").get_attribute("innerHTML")
    address = address.split("<")[0]

    # telephone
    telephones = driver.find_element_by_xpath("//div[@class='col-md-11']/span[@class='company_data_point' and @name='company_phone']/span").get_attribute("innerHTML")

    # website
    website = driver.find_element_by_xpath(
        "//div[@class='col-md-11']/span[@class='company_data_point' and @name='company_website']/span").get_attribute("innerHTML")
    website = website.split(">")[1].split("<")[0]
    
    # employees
    employees = driver.find_element_by_xpath(
        "//div[@class='col-md-11']/span[@class='company_data_point' and @name='employees_all_site']/span").get_attribute("innerHTML")

    # revenue
    revenue = driver.find_element_by_xpath(
        "//div[@class='col-md-11']/span[@class='company_data_point' and @name='revenue_in_us_dollar']/span").get_attribute("innerHTML")

    # name
    name = driver.find_element_by_xpath(
        "//div[@class='margin-between-rows']/span[@class='company_data_point' and @name='company_name']/span").get_attribute("innerHTML")

    # description
    description = driver.find_element_by_xpath(
        "//div[@class='margin-between-rows']/span[@class='company_data_point' and @name='company_description']/span").get_attribute("innerHTML")

    # contact person
    contact_person = driver.find_element_by_xpath(
        "//div[@class='margin-between-rows']/span[@class='company_data_point' and @name='key_principal']/span").get_attribute("innerHTML")
    contact_person = contact_person.split("&")[0]

    
    # services
    services = driver.find_element_by_xpath(
        "//div[@class='margin-between-rows']/span[@class='company_data_point' and @name='industry_links']/span").get_attribute("innerHTML")

    TAG_RE = re.compile(r'<[^>]+>')
    def remove_tags(text):
        return TAG_RE.sub('', text)


    services = remove_tags(services).replace("\n", " ").replace("&nbsp;", "").replace(",", "")

    driver.close()

    df = pd.DataFrame(columns = [
        "name", "industry", "description", "naics", "services", "website", 
        "employees", "contact_person", "address", ""])

# Outpatient Care Centers Companies in Canada


In [152]:
# get naics
naics = get_naics()

6214


In [153]:
dnd_links = get_dnd_links()
len(dnd_links)

50

In [ ]:
## dnd
# names, physical location, sales revenue($M)
cap = DesiredCapabilities().FIREFOX
cap["marionette"] = False
driver = webdriver.Firefox(capabilities=cap, executable_path="/home/ada/Downloads/geckodriver")

driver.get('https://www.dnb.com/business-directory/company-information.health_care_and_social_assistance.ca.html?page=2')

# links = driver.find_elements_by_class_name("col-md-12 data")
vals = driver.find_element(By.TAG_NAME, 'div')
vals = vals.find_elements(By.ID, value="companyResults")
print('length of links are: ', len(vals))
names = []
for i in range(len(vals)):
    names.append(vals[i].text)


driver.close()